#初始化 
#輸入的資料是從2020/3/5開始

In [27]:
#from scipy.integrate import z
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import datetime as dt
%matplotlib inline 
!pip install mpld3
import mpld3
mpld3.enable_notebook()
!pip install lmfit
from scipy.integrate import odeint
from scipy.optimize import minimize

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
#4/1AX4XfWjJnpuRtu9h4GiFGRLz7TUupFKo6SUOVD4M-vRe7lBtNBiiBRrlsiQ

Mounted at /content/gdrive


In [ ]:
import pandas as pd
#all_data = pd.read_csv("owid-covid-data_0711.csv")
all_data = pd.read_csv('/content/gdrive/My Drive/owid-covid-data_0711.csv')
all_data = all_data.fillna(0)
Italy = list(all_data.total_cases[83935:84427])
Italy_death = list(all_data.total_deaths[83935:84427])
Italy_hospital = list(all_data.hosp_patients[83935:84427])
print(len(Italy))
t = np.linspace(0, 450, 451)
f, ax = plt.subplots(1,1,figsize=(15,4))
ax.plot(t, Italy_death[:451], 'k', alpha=0.7, linewidth=2, label='Dead')
ax.plot(t, Italy_hospital[:451], 'y', alpha=0.7, linewidth=2, label='Hospitalized')
ax.set_xlabel('Time (days)')
ax.yaxis.set_tick_params(length=0)
ax.xaxis.set_tick_params(length=0)
ax.grid(b=True, which='major', c='w', lw=2, ls='-')
legend = ax.legend(borderpad=2.0)
legend.get_frame().set_alpha(0.5)
for spine in ('top', 'right', 'bottom', 'left'):
    ax.spines[spine].set_visible(False)
plt.show();

f, ax = plt.subplots(1,1,figsize=(15,4))
ax.plot(t, Italy[:451], 'b', alpha=0.7, linewidth=2, label='Total Infected')
ax.set_xlabel('Time (days)')
ax.yaxis.set_tick_params(length=0)
ax.xaxis.set_tick_params(length=0)
ax.grid(b=True, which='major', c='w', lw=2, ls='-')
legend = ax.legend(borderpad=2.0)
legend.get_frame().set_alpha(0.5)
for spine in ('top', 'right', 'bottom', 'left'):
    ax.spines[spine].set_visible(False)
plt.show();

collect_all = []
print(Italy)

In [ ]:
def plot(data):
  f, ax = plt.subplots(1,1,figsize=(15,4))
  ax.plot(list(range(len(data))), data, 'b', alpha=0.7, linewidth=2)
  ax.set_xlabel('Time (days)')
  ax.yaxis.set_tick_params(length=0)
  ax.xaxis.set_tick_params(length=0)
  ax.grid(b=True, which='major', c='w', lw=2, ls='-')
  legend = ax.legend(borderpad=2.0)
  legend.get_frame().set_alpha(0.5)
  for spine in ('top', 'right', 'bottom', 'left'):
      ax.spines[spine].set_visible(False)
  plt.show();

wave1 = (25,200)
wave2 = (250,400)
N = 59622350
days1 = wave1[1] - wave1[0]
days2 = wave2[1] - wave2[0]

TotalInfected_1 = Italy[wave1[0]:wave1[1]]
TotalInfected_2 = Italy[wave2[0]:wave2[1]]
Death_1 = Italy_death[wave1[0]:wave1[1]]
Death_2 = Italy_death[wave2[0]:wave2[1]]

plot(TotalInfected_1)
plot(TotalInfected_2)
plot(Death_1)
plot(Death_2)

#封城模型 (第一波疫情)
2020/5/7 到 2021/3/1

第57~241天 (第二波疫情)

In [131]:
def plot(t, S, I, H, R, D, QS, QI, QR, real_Total_Infected, real_Total_Deaths, days, show = ['S', 'I', 'H', 'R', 'D', 'QS', 'QI', 'QR', 'pTI', 'pTD', 'rTI', 'rTD']):
  predicted_Total_Infected = []
  for i in range(days):
    predicted_Total_Infected.append( H[i]+R[i]+D[i]+QR[i] )
  #計算總感染人數 
  
  if ('S' in show) or ('I' in show) or ('R' in show) or ('H' in show) or ('D' in show) or ('QS' in show) or ('QI' in show) or ('QR' in show):
    f, ax = plt.subplots(1,1,figsize=(10,5))
    if 'S' in show:
      ax.plot(t, S, 'b', alpha=0.7, linewidth=2, label='Susceptible')
    if 'I' in show:
      ax.plot(t, I, 'r', alpha=0.7, linewidth=2, label='Infected')
    if 'H' in show:
      ax.plot(t, H, 'y', alpha=0.7, linewidth=2, label='Hospitalized')
    if 'R' in show:
      ax.plot(t, R, 'g', alpha=0.7, linewidth=2, label='Recovered')
    if 'D' in show:
      ax.plot(t, D, 'k', alpha=0.7, linewidth=2, label='Dead')
    if 'QS' in show:
      ax.plot(t, QS, 'c', alpha=0.7, linewidth=2, label='Susceptible(Q)')
    if 'QI' in show:
      ax.plot(t, QI, 'm', alpha=0.7, linewidth=2, label='Infected(Q)')
    if 'QR' in show:
      ax.plot(t, QR, '#024E9F', alpha=0.7, linewidth=2, label='Recovered(Q)')

    ax.set_xlabel('Time (days)', labelpad=16, size=16)
    ax.set_ylabel('Populations (people)', labelpad=16, size=16)

    ax.yaxis.set_tick_params(length=0)
    ax.xaxis.set_tick_params(length=0)
    ax.grid(b=True, which='major', c='w', lw=2, ls='-')
    legend = ax.legend(borderpad=2.0)
    legend.get_frame().set_alpha(0.5)
    for spine in ('top', 'right', 'bottom', 'left'):
        ax.spines[spine].set_visible(False)
    plt.show();

  # 針對總感染人數與總死亡人數的數據作圖
  if ('rTI' in show) or ('rTD' in show) or ('pTI' in show) or ('pTD' in show):
    f1, ax1 = plt.subplots(1,1,figsize=(10,5))

    if 'rTI' in show:
      ax1.plot(t, real_Total_Infected, 'b--', alpha=0.7, linewidth=2, label='Real Total Infected')
    if 'rTD' in show:
      ax1.plot(t, real_Total_Deaths, 'k--', alpha=0.7, linewidth=2, label='Real Total Deaths')
    
    if 'pTI' in show:
      ax1.plot(t, predicted_Total_Infected, 'b', alpha=0.7, linewidth=2, label='Predicted Total Infected')  
    if 'pTD' in show:
      ax1.plot(t, D, 'k', alpha=0.7, linewidth=2, label='Predicted Total Deaths')

    ax1.set_xlabel('Time (days)', labelpad=16, size=16)
    ax1.set_ylabel('Populations (people)', labelpad=16, size=16)

    ax1.yaxis.set_tick_params(length=0)
    ax1.xaxis.set_tick_params(length=0)
    ax1.grid(b=True, which='major', c='w', lw=2, ls='-')
    legend = ax1.legend()
    legend.get_frame().set_alpha(0.5)
    for spine in ('top', 'right', 'bottom', 'left'):
      ax1.spines[spine].set_visible(False)
    plt.show();

In [102]:
def get_diff(populations, paras):
  (S, I, H, R, D, QS, QI, QR) = populations
  (alpha, beta, r_I, r_H, eta, tau, mu) = paras

  n = S + I + R

  dS = - alpha * S * I / n - S*tau*(beta*I+beta*QI) + mu*QS
  dI = alpha * S * I / n - beta * I - r_I *I - I*tau*(beta*I+beta*QI)
  dH = beta * I - r_H * H - eta * H + beta*QI
  dR = r_I *I + r_H * H - R*tau*(beta*I+beta*QI) + mu*QR
  dD = eta * H
  dQS = S*tau*(beta*I+beta*QI) - mu*QS
  dQI = I*tau*(beta*I+beta*QI) - r_I*QI - beta*QI 
  dQR = R*tau*(beta*I+beta*QI) + r_I*QI - mu*QR
  return (dS, dI, dH, dR, dD, dQS, dQI, dQR)
  
def mimic(days, initial, paras):
  t = np.linspace(0, days-1, days)
  [S0, I0, H0, R0, D0, QS0, QI0, QR0] = initial

  S = [S0]
  I = [I0]
  H = [H0]
  R = [R0]
  D = [D0]
  QS = [QS0]
  QI = [QI0]
  QR = [QR0]
  for i in range(days-1):
    populations = [S[-1],I[-1],H[-1],R[-1],D[-1],QS[-1],QI[-1],QR[-1]]
    (dS, dI, dH, dR, dD, dQS, dQI, dQR) = get_diff(populations, paras)
    S.append(S[-1]+dS)
    I.append(I[-1]+dI)
    H.append(H[-1]+dH)
    R.append(R[-1]+dR)
    D.append(D[-1]+dD)
    QS.append(QS[-1]+dQS)
    QI.append(QI[-1]+dQI)
    QR.append(QR[-1]+dQR)

  return (t, S, I, H, R, D, QS, QI, QR) #以陣列回傳

In [135]:
#[alpha1, alpha2, beta, r_I, r_H, eta1, eta2, tau, mu] = [0.2272406488288084, 1.69390133896621] + [0.001]*7
#[alpha1, alpha2, beta, r_I, r_H, eta1, eta2, tau, mu] = [0.5240658623087001, 0.6237138881360947, 0.015545150803274382, 0.4371684311473786, 0.4371684311473786, 0.09340477951081945, 0.09340477951081945, 0.00020424242233467476, 9.370566915917284e-47]
#[alpha1, alpha2, beta1, beta2, r_I1, r_I2, r_H1, r_H2, eta1, eta2, tau, mu] = [0.48723469028469774, 0.5858933545068362, 0.01988064149382497, 0.01988064149382497, 0.39407830926111664, 0.39407830926111664, 0.018395730278878468, 0.018395730278878468, 0.025839519255141337, 0.09288786546081026, 0.00014953980756786663, 0.1]
[alpha1, alpha2, beta1, beta2, r_I1, r_I2, r_H1, r_H2, eta1, eta2, tau, mu] = [0.05]*2+[0.5]*8+[0.01]*2
#import warnings
#warnings.filterwarnings("default")

In [ ]:
[H0, QS0, QI0, QR0] = [0]*4

def error_function(parameters):
  global alpha1, alpha2, beta1, beta2, r_I1, r_I2, r_H1, r_H2, eta1, eta2, tau, mu
  alpha1, alpha2, beta1, beta2, r_I1, r_I2, r_H1, r_H2, eta1, eta2, tau, mu = parameters

  error = 0
  for i in parameters:
    error += int(i<0.0000)*(1-i)**2*1e15
  error += int(alpha1>alpha2)*(alpha1-alpha2)**2*1e15
  error += int(r_H1<r_I1*0.01)*(r_I1-r_H1)**2*1e15
  error += int(r_H2<r_I2*0.01)*(r_I2-r_H2)**2*1e15
  error += int(mu<0.01)*(1-mu)**2*1e15

  R0 = TotalInfected_1[0] - Death_1[0]
  I0 = 100
  D0 = Death_1[0]
  S0 = N - R0 - I0 - D0
  (t, S, I, H, R, D, QS, QI, QR) = mimic(days1, [S0, I0, H0, R0, D0, QS0, QI0, QR0], [alpha1, beta1, r_I1, r_H1, eta1, tau, mu])
  predicted_TotalInfected_1 = []
  for i in range(days1):
    predicted_TotalInfected_1.append(H[i]+R[i]+D[i]+QR[i])
  for i in range(days1):
    error += (TotalInfected_1[i]-predicted_TotalInfected_1[i])**2
    error += (Death_1[i]-D[i])**2 *50
  
  R0 = TotalInfected_2[0] - Death_2[0]
  I0 = 100
  D0 = Death_2[0]
  S0 = N - R0 - I0 - D0
  (t, S, I, H, R, D, QS, QI, QR) = mimic(days2, [S0, I0, H0, R0, D0, QS0, QI0, QR0], [alpha2, beta2, r_I2, r_H2, eta2, tau, mu])
  predicted_TotalInfected_2 = []
  for i in range(days2):
    predicted_TotalInfected_2.append(H[i]+R[i]+D[i]+QR[i])  
  for i in range(days2):
    error += (TotalInfected_2[i]-predicted_TotalInfected_2[i])**2
    error += (Death_2[i]-D[i])**2 *50
  
  return error

for i in range(10):
  Result = minimize(error_function, [alpha1, alpha2, beta1, beta2, r_I1, r_I2, r_H1, r_H2, eta1, eta2, tau, mu], method = 'Nelder-Mead', options={'maxiter':1e3})
  print(Result.fun/1e9, list(Result.x))
print()
print(Result)
[alpha1, alpha2, beta1, beta2, r_I1, r_I2, r_H1, r_H2, eta1, eta2, tau, mu] = list(Result.x)

#0.3613914607935472 [0.5240658623087001, 0.6237138881360947, 0.015545150803274382, 0.4371684311473786, 1.8120009952029614e-63, 0.09340477951081945, 0.00020424242233467476, 9.370566915917284e-47]
#2.08201560677332 [0.19917010931141044, 0.45632631220048914, 0.0001612602546171412, 0.0003376799973398343, 0.00010000000093987659, 0.0015603919301444007, 0.003263617150941586, 0.00010000000004869381, 0.00014732039660217769]
#0.9981266364368617 [0.2272406488288084, 1.69390133896621, 6.182439400316338e-06, 0.00014366979837924094, 3.388350016063105e-18, 0.02001179375122896, 0.12571821220432006, 0.00019170364600160019, 3.0190557136958305e-16]
#0.2123296416135527 [0.48723469028469774, 0.5858933545068362, 0.01988064149382497, 0.39407830926111664, 0.018395730278878468, 0.025839519255141337, 0.09288786546081026, 0.00014953980756786663, 4.4338012696009044e-49]
#562.2235297366656 [0.13906397560524864, 1.1975288609511665, 0.0006610256427758247, 0.04858044129968728, 0.02048497167853619, 0.7928682852925184, 0.010188524313668186, 0.04150771415413315, 0.05045959451318657, 0.08753276224791284, 0.0006322999575467825, 1.7784778796069949]

In [ ]:
[H0, QS0, QI0, QR0] = [0]*4

R0 = TotalInfected_1[0] - Death_1[0]
I0 = 100
D0 = Death_1[0]
S0 = N - R0 - I0 - D0
(t, S, I, H, R, D, QS, QI, QR) = mimic(days1, [S0, I0, H0, R0, D0, QS0, QI0, QR0], [alpha1, beta1, r_I1, r_H1, eta1, tau, mu])
plot(t, S, I, H, R, D, QS, QI, QR, TotalInfected_1, Death_1, days1, show = ['S', 'I', 'H', 'R', 'D', 'QS', 'QI', 'QR', 'pTI', 'pTD', 'rTI', 'rTD'])


R0 = TotalInfected_2[0] - Death_2[0]
I0 = 100
D0 = Death_2[0]
S0 = N - R0 - I0 - D0
(t, S, I, H, R, D, QS, QI, QR) = mimic(days2, [S0, I0, H0, R0, D0, QS0, QI0, QR0], [alpha2, beta2, r_I2, r_H2, eta2, tau, mu])
plot(t, S, I, H, R, D, QS, QI, QR, TotalInfected_2, Death_2, days2, show = ['S', 'I', 'H', 'R', 'D', 'QS', 'QI', 'QR', 'pTI', 'pTD', 'rTI', 'rTD'])